In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

## 리뷰 전처리

In [5]:
df=pd.read_csv('../preprocess/top10000_review_preprocess.csv', encoding='utf-8')

In [6]:
df['review_text']=df['review_text'].str.replace("[^ㄱ-하-ㅣ가-힣 ]","")

C:\Users\duwjd\AppData\Local\Temp\ipykernel_12896\2856999036.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_text']=df['review_text'].str.replace("[^ㄱ-하-ㅣ가-힣 ]","")


In [7]:
reviews=df.loc[:,'review_text'].unique()
train_data=pd.DataFrame({'document':reviews})
train_data

,document
0,배송 이 빨라 좋아요 당일 출고 되 네요 감사합니다
1,배송도 빠르고 가방동 마음에 들어요 잘쓰겠습니다
2,배송빨라요정사이즈에요착용감좋아요편안해요
3,배송이 빨라요수납력 엄청납니다착용감 좋아요편안해요
4,배송빨라요수납력이 엄청나요착용감좋아요편안해요
...,...
1066568,무신사 이벤트때 착한가격으로 잘 구매했어요감사합니다
1066569,안녕하세요버뮤다 핑크 이쁘네요잘 구매했어요감사합니다
1066570,무신사에서 좋은제품 사게되어서 좋습니다다음에도 살게요
1066571,생로랑핏에 맞게 세쿠시하게 입기보다는 과잠처럼 여유있게 입는게 좋아서 일주러 크게 ...


In [51]:
from sklearn.utils import shuffle
a = shuffle(train_data, random_state=0).head(10000)

In [57]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

# Vectorize text
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(a['document'])

# Dimensionality reduction
pca = PCA(n_components=50)  # Adjust as needed
X_reduced = pca.fit_transform(X.toarray())

num_clusters = 5000
minibatch_kmeans = MiniBatchKMeans(n_clusters=num_clusters, n_init=10, random_state=42)
clusters = minibatch_kmeans.fit_predict(X_reduced)

# Assign clusters to DataFrame
a['cluster'] = clusters

# Inspect results
for cluster_num in range(num_clusters):
    print(f"\nCluster {cluster_num}:")
    print(a[a['cluster'] == cluster_num]['document'].values)


Cluster 0:
['딱 맞는 하의 찾기 어려운데수선없이 딱 맞아서 넘 좋아요' '심플하면서 가벼워요크기도 적당하고무엇보다 수납공간이 넉넉해서 좋아요'
 '면퍼 아니라 좋아요 차르르 한재질이라  맘에듭니디ㅡ']

Cluster 1:
['새로운 향수 찾다가 매장에서 맡아보고 너무 마음에 들어서 바로 구매하고 혹시나 무신사에도 있을까해서 찾아보니깐 있어서 기쁜 마음으로 구매했습니다일단 향이 달달하고 너무 쎄지 않아서 너무 좋고향 지속시간은 주관적으로 -시간 정도 같아요남녀 둘다 좋아할만하고 친구들도 무슨 향 쓰냐고 물어봅니다비건향수라고 하는데 저한텐 딱히 의미가 없어요부담스럽지 않은 향 찾으시는 분들한테 추천할게요'
 '약간 광택이 있어서 빤질빤질하네요핏은 너무 이쁩니다요래조래 코디해서 잘 입는중'
 '다른 컬러 원턱이 있어서 산건데다른건 다 괜찮은데 기장이 너무 짧아서 너무 아쉽습니다 사진은 어깨부터 전신인데 제대로 확인하시고미충족이라고 해주신건 맞는거죠'
 '세일할때 구입했어요숏 롱으로 나뉘어져 있어서 저는 다리가 짧아 숏으로 했구요크롭과 세트로 입으니 잘 맞고 예쁘네요천도 너무 구김 잘 가는 재질은 아니고 편안하게 그렇다고 너무 츄리닝 같아 보이지도 않아서 좋으네요블랙이랑 두 가지 주문 해서 입었어요'
 '사이즈감이 너무 크고 촉감이 거칠어요내피가 있어서 확실히 따듯하긴해요'
 '디자인은 너무 얘뻐서 일년 전부터 사고 싶었는데 할인이 있어서 사게 됐어요']

Cluster 2:
['너무너무좋아용 강력추천합니다 또재구매할게요 너무만족합니다' '지인들이잘샀다고하네요ㅎㅎ 좋은가격에잘구매했네요 추천추천 최고최고']

Cluster 3:
['디퓨저를 사랑하는 인입니다 여기 제품 너무 고급져요 향도 고급지고 포장도 고급지고 다 고급져요 디자인도 예쁘고 참 기분 좋아지는 향이네요 디퓨저는 사랑입니다 저희집은 항상 디퓨저를 쟁여놓고 쓰고 있어요 우리집만의 향을 만들려고요 스윗베리향으로 우리집을 가득 채울꺼에요 고급지고 제 스타일이에요 너무 마음에 들어요향은 은은하게 퍼

In [77]:
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

def get_top_n_terms(texts, n=10):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    X = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = X.mean(axis=0).A1
    top_indices = tfidf_scores.argsort()[-n:][::-1]
    top_terms = [feature_names[i] for i in top_indices]
    return top_terms

cluster_sentences = defaultdict(list)
for idx, row in a.iterrows():
    cluster_sentences[row['cluster']].append(row['document'])

# Number of terms to extract
num_terms = 10  # Adjust as needed

# Summarize each cluster
cluster_summary = {}
for cluster in sorted(cluster_sentences.keys()):
    sentences = cluster_sentences[cluster]
    summary_terms = get_top_n_terms(sentences, n=num_terms)
    cluster_summary[cluster] = summary_terms

In [93]:
li=[]
for cluster, terms in sorted(cluster_summary.items()):
        li.append(' '.join(terms))

In [95]:
aa=pd.DataFrame({'document':li})
aa

,document
0,좋아요 한재질이라 하의 찾기 차르르 적당하고무엇보다 어려운데수선없이 아니라 심플하면...
1,너무 있어서 크고 입는중 확실히 약간 거칠어요내피가 광택이 사이즈감이 따듯하긴해요
2,추천추천 최고최고 지인들이잘샀다고하네요ㅎㅎ 좋은가격에잘구매했네요 또재구매할게요 너무...
3,고급지고 고급져요 너무 디퓨저를 향이네요 디퓨저는 스틱도 스타일이에요 스윗베리향으로...
4,빠르고 배송도 포장도 사용했던 거여서 예전부터 기본탬으로는 이만한신발이 좋네요ㅎㅎ ...
...,...
4536,핑크로 특이점없이 잘신고 있어요 있어서 일반양말에 예쁩니다 않고 세일가격에 샀어요
4537,같아요 한여름엔 입으면 이쁠거 선선할때 생각보다 못입을거 두꺼워요 더워서
4538,맘에 친구 줬는데 선물로 두꺼워요 선물했는데 들오요 들우하네요 재구매했오요여전히 따뜻하고
4539,편해요 토트로 강추 강할지는 강해서 같아요 같아요ㅎㅎ 것보다 기본 당분간


### 맞춤법 라이브러리

In [ ]:
#!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import spell_checker
spell=[]
for i in tqdm(a['document']):
    sent = i
    spelled_sent = spell_checker.check(sent)
    hanspell_sent = spelled_sent.checked
    spell.append(hanspell_sent)

a['spell_doc']=spell
a

### 띄어쓰기 라이브러리

In [ ]:
#pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [97]:
from pykospacing import Spacing
spac=[]
for i in tqdm(aa['document']):
    spacing = Spacing()
    kospacing_result = spacing(i) 
    spac.append(kospacing_result)

aa['spac_doc']=spac
aa

100%|██████████████████████████████████████████████████████████████████████████████| 4541/4541 [03:22<00:00, 22.44it/s]


,document,spac_doc
0,좋아요 한재질이라 하의 찾기 차르르 적당하고무엇보다 어려운데수선없이 아니라 심플하면...,좋아요 한 재질이라 하의 찾기 차르르 적당하고 무엇보다 어려운데 수선 없이 아니라 ...
1,너무 있어서 크고 입는중 확실히 약간 거칠어요내피가 광택이 사이즈감이 따듯하긴해요,너무 있어서 크고 입는 중 확실히 약간 거칠어요 내피가 광택이 사이즈감이 따듯하긴 해요
2,추천추천 최고최고 지인들이잘샀다고하네요ㅎㅎ 좋은가격에잘구매했네요 또재구매할게요 너무...,추천추천 최고 최고 지인들이 잘 샀다고 하네요 ㅎㅎ 좋은 가격에 잘 구매했네요 또 ...
3,고급지고 고급져요 너무 디퓨저를 향이네요 디퓨저는 스틱도 스타일이에요 스윗베리향으로...,고급 지고 고급져요 너무 디퓨저를 향이네요 디퓨저는 스틱도 스타일이에 요 스윗베리 ...
4,빠르고 배송도 포장도 사용했던 거여서 예전부터 기본탬으로는 이만한신발이 좋네요ㅎㅎ ...,빠르고 배송도 포장도 사용했던 거여서 예전부터 기 본탬으로는 이만한 신발이 좋네요 ...
...,...,...
4536,핑크로 특이점없이 잘신고 있어요 있어서 일반양말에 예쁩니다 않고 세일가격에 샀어요,핑크로 특이점 없이 잘 신고 있어요 있어서 일반 양말에 예쁩니다 않고 세일 가격에 샀어요
4537,같아요 한여름엔 입으면 이쁠거 선선할때 생각보다 못입을거 두꺼워요 더워서,같아 요 한 여름엔 입으면 이쁠 거 선선할 때 생각보다 못 입을 거 두꺼워 요 더워서
4538,맘에 친구 줬는데 선물로 두꺼워요 선물했는데 들오요 들우하네요 재구매했오요여전히 따뜻하고,맘에 친구 줬는데 선물로 두꺼워요 선물했는데 들 오요 들 우하네요 재구매했오요 여전...
4539,편해요 토트로 강추 강할지는 강해서 같아요 같아요ㅎㅎ 것보다 기본 당분간,편해요 토트로 강추 강할지는 강해서 같아 요 같아 요ㅎㅎ 것보다 기본 당분간


### 사용자 단어 추가

In [ ]:
# import zipfile
# import os
# import shutil

# # Define paths
# jar_path = r'C:\Users\duwjd\anaconda3\Lib\site-packages\konlpy\java\open-korean-text-2.1.0.jar'
# extract_dir = r'C:\Users\duwjd\anaconda3\Lib\site-packages\konlpy\java\extracted'
# new_jar_path = r'C:\Users\duwjd\anaconda3\Lib\site-packages\konlpy\java\open-korean-text-2.1.0-modified.jar'
# backup_jar_path = r'C:\Users\duwjd\anaconda3\Lib\site-packages\konlpy\java\open-korean-text-2.1.0-backup.jar'

# # Ensure the extraction directory exists
# os.makedirs(extract_dir, exist_ok=True)

# # Extract the JAR file
# with zipfile.ZipFile(jar_path, 'r') as jar:
#     jar.extractall(extract_dir)

# # Path to the specific file to modify
# file_path = os.path.join(extract_dir, 'org/openkoreantext/processor/util/noun/names.txt')

# # Read the contents of the file
# with open(file_path, 'r', encoding='utf-8') as f:
#     data = f.read()

# # Add new words to the dictionary
# data += '강 추\n요정 사이즈\n스타 일링\n'

# # Write the updated data back to the file
# with open(file_path, 'w', encoding='utf-8') as f:
#     f.write(data)

# # Recompress the modified files into a new JAR file
# with zipfile.ZipFile(new_jar_path, 'w') as new_jar:
#     for root, dirs, files in os.walk(extract_dir):
#         for file in files:
#             file_path = os.path.join(root, file)
#             arcname = os.path.relpath(file_path, extract_dir)
#             new_jar.write(file_path, arcname)

# # Optional cleanup: remove the extracted directory
# shutil.rmtree(extract_dir)

# # Attempt to replace the original JAR with the modified one
# try:
#     # Rename the original JAR to a backup
#     os.rename(jar_path, backup_jar_path)

#     # Move the modified JAR to the original location
#     shutil.move(new_jar_path, jar_path)

#     # If successful, remove the backup JAR
#     os.remove(backup_jar_path)

# except PermissionError as e:
#     print(f"PermissionError: {e}")
#     print("Please ensure you have the necessary permissions to modify the JAR file and try again.")
#     print("Alternatively, you can manually replace the original JAR file with the modified one located at:")
#     print(new_jar_path)
#     print(f"and rename the original JAR file at {jar_path} to keep a backup.")

## 토큰화

In [99]:
from collections import Counter

# 불용어 정의
with open('stopwords.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")

okt = Okt()

# Tokenize and remove stopwords
tokenized_data = []
for sentence in tqdm(aa['spac_doc']):
    tokenized_sentence = okt.morphs(sentence, stem=True)  # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords and len(word) >= 2]  # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

# Count token frequencies
token_counter = Counter(token for sentence in tokenized_data for token in sentence)

# Filter tokens by frequency
filtered_tokenized_data = []
for sentence in tokenized_data:
    filtered_sentence = [word for word in sentence if token_counter[word] > 3]
    filtered_tokenized_data.append(filtered_sentence)

# Assign filtered data back to the DataFrame
aa['token_doc'] = filtered_tokenized_data
aa

100%|█████████████████████████████████████████████████████████████████████████████| 4541/4541 [00:23<00:00, 190.30it/s]


,document,spac_doc,token_doc
0,좋아요 한재질이라 하의 찾기 차르르 적당하고무엇보다 어려운데수선없이 아니라 심플하면...,좋아요 한 재질이라 하의 찾기 차르르 적당하고 무엇보다 어려운데 수선 없이 아니라 ...,"[좋다, 재질, 하의, 찾기, 적당하다, 어렵다, 수선, 심플, 수납, 공간]"
1,너무 있어서 크고 입는중 확실히 약간 거칠어요내피가 광택이 사이즈감이 따듯하긴해요,너무 있어서 크고 입는 중 확실히 약간 거칠어요 내피가 광택이 사이즈감이 따듯하긴 해요,"[크다, 입다, 확실하다, 거치다, 광택, 사이, 따다]"
2,추천추천 최고최고 지인들이잘샀다고하네요ㅎㅎ 좋은가격에잘구매했네요 또재구매할게요 너무...,추천추천 최고 최고 지인들이 잘 샀다고 하네요 ㅎㅎ 좋은 가격에 잘 구매했네요 또 ...,"[추천, 추천, 최고, 최고, 지인, 자다, 사다, 좋다, 가격, 자다, 구매, 만..."
3,고급지고 고급져요 너무 디퓨저를 향이네요 디퓨저는 스틱도 스타일이에요 스윗베리향으로...,고급 지고 고급져요 너무 디퓨저를 향이네요 디퓨저는 스틱도 스타일이에 요 스윗베리 ...,"[고급, 지고, 고급, 지다, 스타일]"
4,빠르고 배송도 포장도 사용했던 거여서 예전부터 기본탬으로는 이만한신발이 좋네요ㅎㅎ ...,빠르고 배송도 포장도 사용했던 거여서 예전부터 기 본탬으로는 이만한 신발이 좋네요 ...,"[빠르다, 배송, 포장, 사용, 예전, 이만, 신발, 좋다, 니당다]"
...,...,...,...
4536,핑크로 특이점없이 잘신고 있어요 있어서 일반양말에 예쁩니다 않고 세일가격에 샀어요,핑크로 특이점 없이 잘 신고 있어요 있어서 일반 양말에 예쁩니다 않고 세일 가격에 샀어요,"[핑크, 자다, 신고, 일반, 양말, 예쁘다, 가격, 사다]"
4537,같아요 한여름엔 입으면 이쁠거 선선할때 생각보다 못입을거 두꺼워요 더워서,같아 요 한 여름엔 입으면 이쁠 거 선선할 때 생각보다 못 입을 거 두꺼워 요 더워서,"[여름, 입다, 이쁘다, 선선, 생각, 두껍다, 덥다]"
4538,맘에 친구 줬는데 선물로 두꺼워요 선물했는데 들오요 들우하네요 재구매했오요여전히 따뜻하고,맘에 친구 줬는데 선물로 두꺼워요 선물했는데 들 오요 들 우하네요 재구매했오요 여전...,"[친구, 선물, 두껍다, 선물, 따뜻하다]"
4539,편해요 토트로 강추 강할지는 강해서 같아요 같아요ㅎㅎ 것보다 기본 당분간,편해요 토트로 강추 강할지는 강해서 같아 요 같아 요ㅎㅎ 것보다 기본 당분간,"[편하다, 강추, 강하다, 강하다, 기본]"


In [139]:
token_counter = Counter(token for sentence in aa['token_doc'] for token in sentence)
len(token_counter)

1042

In [197]:
replacements = {
    '버핏' : '오버핏',
    '빨르다' : '빠르다',
    '디테' : '디테일',
    '이쁘다' : '예쁘다',
    '쁘네' : '예쁘네',
    '휘뚜루' : '휘뚜루',
    '뚜루' : '휘뚜루',
    '휘뚤마뚤' : '휘뚜루',
    '성도' : '완성도',
    '두운' : '어두운',
    '투맨' : '맨투맨',
    '조아' : '좋다',
    '레터' : '레터링',
    '웨이' : '투웨이',
    '데일' : '데일리',
    '쁜데' : '예쁜데',
    '쁩니' : '예쁩니다',
    '핏입니' : '핏입니다',
    '가성' : '가성비',
    
}

# Function to replace tokens based on the replacements dictionary
def replace_tokens(tokens, replacements):
    return [replacements.get(token, token) for token in tokens]

# Apply function to 'token' column
aa['token_doc'] = aa['token_doc'].apply(lambda x: replace_tokens(x, replacements))
aa

,document,spac_doc,token_doc
0,좋아요 한재질이라 하의 찾기 차르르 적당하고무엇보다 어려운데수선없이 아니라 심플하면...,좋아요 한 재질이라 하의 찾기 차르르 적당하고 무엇보다 어려운데 수선 없이 아니라 ...,"[좋다, 재질, 하의, 찾기, 적당하다, 어렵다, 수선, 심플, 수납, 공간]"
1,너무 있어서 크고 입는중 확실히 약간 거칠어요내피가 광택이 사이즈감이 따듯하긴해요,너무 있어서 크고 입는 중 확실히 약간 거칠어요 내피가 광택이 사이즈감이 따듯하긴 해요,"[크다, 입다, 확실하다, 거치다, 광택, 사이, 따다]"
2,추천추천 최고최고 지인들이잘샀다고하네요ㅎㅎ 좋은가격에잘구매했네요 또재구매할게요 너무...,추천추천 최고 최고 지인들이 잘 샀다고 하네요 ㅎㅎ 좋은 가격에 잘 구매했네요 또 ...,"[추천, 추천, 최고, 최고, 지인, 자다, 사다, 좋다, 가격, 자다, 구매, 만..."
3,고급지고 고급져요 너무 디퓨저를 향이네요 디퓨저는 스틱도 스타일이에요 스윗베리향으로...,고급 지고 고급져요 너무 디퓨저를 향이네요 디퓨저는 스틱도 스타일이에 요 스윗베리 ...,"[고급, 지고, 고급, 지다, 스타일]"
4,빠르고 배송도 포장도 사용했던 거여서 예전부터 기본탬으로는 이만한신발이 좋네요ㅎㅎ ...,빠르고 배송도 포장도 사용했던 거여서 예전부터 기 본탬으로는 이만한 신발이 좋네요 ...,"[빠르다, 배송, 포장, 사용, 예전, 이만, 신발, 좋다, 니당다]"
...,...,...,...
4536,핑크로 특이점없이 잘신고 있어요 있어서 일반양말에 예쁩니다 않고 세일가격에 샀어요,핑크로 특이점 없이 잘 신고 있어요 있어서 일반 양말에 예쁩니다 않고 세일 가격에 샀어요,"[핑크, 자다, 신고, 일반, 양말, 예쁘다, 가격, 사다]"
4537,같아요 한여름엔 입으면 이쁠거 선선할때 생각보다 못입을거 두꺼워요 더워서,같아 요 한 여름엔 입으면 이쁠 거 선선할 때 생각보다 못 입을 거 두꺼워 요 더워서,"[여름, 입다, 예쁘다, 선선, 생각, 두껍다, 덥다]"
4538,맘에 친구 줬는데 선물로 두꺼워요 선물했는데 들오요 들우하네요 재구매했오요여전히 따뜻하고,맘에 친구 줬는데 선물로 두꺼워요 선물했는데 들 오요 들 우하네요 재구매했오요 여전...,"[친구, 선물, 두껍다, 선물, 따뜻하다]"
4539,편해요 토트로 강추 강할지는 강해서 같아요 같아요ㅎㅎ 것보다 기본 당분간,편해요 토트로 강추 강할지는 강해서 같아 요 같아 요ㅎㅎ 것보다 기본 당분간,"[편하다, 강추, 강하다, 강하다, 기본]"


In [203]:
aa.to_csv('token_review.csv', encoding='utf-8')